# League of legend Winner team predict to Using Riot API

2019 데이터분석 프로젝트 리메이크

리그 오브 레전드 경기 기록을 수집하고, 가공하여 승률에 영향을 주는 요인 분석과 실시간 스코어 변동에 따른 승률 예측 프로젝트

A project to collect league-of-legend game records to analyze the factors that affect victory and to predict winning rates based on real-time score fluctuations.

This kernel is for Koreans and I apologize to foreign viewers.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/win_team_stats.csv
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/LOL_predict_cbr.cbm
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_lose_data.pkl
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/matchpre.pkl
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/LOL_predict_xgb.bst
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lol_version_Date.csv
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_ver1.pkl
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_winner_data.pkl
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/stats.csv
/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lose_team_stats.csv


# 1. ETL(Extract,Transform,Load) 데이터 수집 / 전처리 / 적재

## 라이브러리 로드

In [2]:
import pickle #데이터 저장용
import json
import re 
import time
from pandas.io.json import json_normalize
from tqdm import tqdm_notebook # 오래 걸리는 작업 진행확인용
import requests

## DATA PIPELINE

Riot api에는 바로 최근 이뤄진 경기 목록과 기록을 불러오는 api가 존재하지 않는다. 그래도 게임 기록에 접근하는 방법이 있는데,
유저 이름으로 유저 정보를 받아온 후 해당 유저의 게임 목록을 불러오고, 목록의 게임 기록에 접근할 수 있다. 

라이엇 api는 요청 제한이 있고, api_key에 제한 시간이 있으므로 초과하지 않도록 유의하며 모은 데이터를 파일화하여 관리.

제한을 넘기거나, 유효시간이 초과된 api_key로 응답요청을 보내면 블랙리스트에 올라 한동안 데이터 수집이 불가하다.

> 데이터 수집 시간이 오래걸리므로 데이터를 파일화하여 저장한 후 코드들은 주석처리

![image](https://i.imgur.com/3Ktnr0V.png)

## 승패 예측에 필요한 경기 기록을 가져오기 위한 API

1. 소환사 목록 api

모든 티어의 유저 명단이 있다면 좋겠지만 하위 티어는 유저량과 휴면 유저가 많아 Riot은 챌린저, 마스터, 그랜드마스터의 유저명단만을 제공한다.

https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key

2. 소환사 정보 api

유저 명단의 summoner_id를 이용해 해당 유저의 정보를 알 수 있다.

https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key

3. 소환사 전적 api

소환사 정보의 account_id를 이용해 해당 유저의 전적을 불러온다.

https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + '?season=' + season + '&api_key=' + api_key

4. 전적별 게임 기록 api

전적 목록의 game_id를 이용해 해당 게임의 기록을 불러온다.

https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key

## Challenger, grandmaster, master user list
챌린저, 그랜드마스터, 마스터 티어의 유저 명단

In [3]:
"""
Match data 를 가져오기 위해선 먼저 각 소환사의 아이디가 필요하다. 따라서 챌린저, 그랜드마스터, 마스터에 소속된 유저명단을 불러온다.
"""
# api_key = "Riot developer's api key"

# api_challenger = "https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
# api_grandmaster = "https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
# api_master = "https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
# r = requests.get(api_challenger)
# league = pd.DataFrame(r.json())
# r = requests.get(api_grandmaster)
# league = pd.concat([league, pd.DataFrame(r.json())], axis=0)
# r = requests.get(api_master)
# league = pd.concat([league, pd.DataFrame(r.json())], axis=0)

# league.reset_index(inplace=True)
# league_entries = pd.DataFrame(dict(league["entries"])).T
# league = pd.concat([league, league_entries], axis=1)

# league = league.drop(["index", "queue", "name", "leagueId", "entries", "rank"], axis=1) #필요없는 칼럼 드랍

'\nMatch data 를 가져오기 위해선 먼저 각 소환사의 아이디가 필요하다. 따라서 챌린저, 그랜드마스터, 마스터에 소속된 유저명단을 불러온다.\n'

## User info

In [4]:
"""
불러온 유저정보의 summonerId를 이용해 소환사의 정보api에서 account_id를 불러온다
API는 초당 20번, 분당 50번으로 요청 제한이 있으므로 블랙리스트에 오르지 않도록 유의해서 데이터를 수집하고 파일로 저장.
"""
# league["account_id"] = np.nan # account_id 초기화
# for i, summoner_id in enumerate(league["summonerId"]):
#     #Summoner API에서 AccountId를 가져와 채워넣는다.
#     api_url = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/" + summoner_id + "?api_key=" + api_key
#     r = requests.get(api_url)
#     while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
#         time.sleep(5)
#         r = requests.get(api_url)
#     account_id = r.json()["accountId"]
#     league.iloc[i, -1] = account_id

# league.to_csv("LeagueData.csv")

'\n불러온 유저정보의 summonerId를 이용해 소환사의 정보api에서 account_id를 불러온다\nAPI는 초당 20번, 분당 50번으로 요청 제한이 있으므로 블랙리스트에 오르지 않도록 유의해서 데이터를 수집하고 파일로 저장.\n'

## Match list

In [5]:
"""
소환사의 account_id 로부터 치뤄졌던 경기들의 목록인 Matchlist를 불러온다.
"""

# season = str(13) #13시즌에 치뤄진 경기를 불러온다.

# match_info_df = pd.DataFrame()
# for account_id in league_df["account_id"]:
#     api_url = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/" + account_id + \
#                   "?season=" + season + "&api_key=" + api_key
#     r = requests.get(api_url)
#     while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
#         time.sleep(5)
#         r = requests.get(api_url)
#     match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()["matches"])])
# match_info_df.to_csv("MatchInfoData.csv")

'\n소환사의 account_id 로부터 치뤄졌던 경기들의 목록인 Matchlist를 불러온다.\n'

## Match info

In [6]:
"""
Matchlist에서 얻은 경기 한 판의 키값인 gameId로 해당 판의 기록을 불러온다.
"""

# match_info_df = match_info_df.drop_duplicates("gameId") #중복 경기기록은 삭제

# match_df = pd.DataFrame()
# for game_id in match_info_df["gameId"]:
#     api_url = "https://kr.api.riotgames.com/lol/match/v4/matches/" + str(game_id) + "?api_key=" + api_key
#     r = requests.get(api_url)
#     while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
#         time.sleep(5)
#         r = requests.get(api_url)
#     r_json = r.json()
#     temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T
#     match_df = pd.concat([match_df, temp_df])

# match_df.to_pickle("Match.pkl") # 파일로 저장

'\nMatchlist에서 얻은 경기 한 판의 키값인 gameId로 해당 판의 기록을 불러온다.\n'

경기목록과 경기내용은 API 제한 때문에 수집이 굉장히 느리다. time모듈로 끊어서 수집하였고 9만개의 데이터를 확보하기까지 18일정도가 걸렸다.

(하루 5시간 수집)

## Add Date Column

버전 정보만으로 인사이트를 얻어내기에 어려움이 있어 롤 패치버전과 날짜가 기재된 외부 사이트를 크롤링해 날짜 칼럼 추가

In [7]:
# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import urllib
# import csv
# import re

Reffered to <https://whatpulse.org/app/league-of-legends>

In [8]:
"""
해당 사이트의 html 테이블 구조가 이상해 find 메서드가 제대로 작동 안함에 따라
odd,even,date를 find_all 따로 수집후 한번에 병합
"""
# url = "https://whatpulse.org/app/league-of-legends"
# html = urlopen(url)
# obj = BeautifulSoup(html.read(),"html.parser")

# temp = list(obj.find_all("td",{"align":"center"}))

# date = []
# for i in range(len(temp)):
#     if(i%2==1):
#         datei = temp[i].text
#         date.append(datei)

# tempodd = list(obj.find_all("tr",{"class":"odd"}))
# tempeven = list(obj.find_all("tr",{"class":"even"}))

# tempodd2 = []
# for i in range(len(tempodd)):
#     tempi = tempodd[i].text
#     tempodd2.append(tempi)

# tempeven2 = []
# for i in range(len(tempeven)):
#     tempi = tempeven[i].text
#     tempeven2.append(tempi)

# version = []
# for i in range(len(tempodd2)):
#     version.append(tempodd2[i])
#     version.append(tempeven2[i])


# version_date =  pd.DataFrame(data={"version":version,"date":date})

# version_date['gameVersion'] = version_date['version'].str.replace(pat=r'[^0-9]', repl=r'', regex=True)

# version_date = version_date[["date","gameVersion"]]

# version_date.to_csv("lol_version_Date.csv",index=False)

'\n해당 사이트의 html 테이블 구조가 이상해 find 메서드가 제대로 작동 안함에 따라\nodd,even,date를 find_all 따로 수집후 한번에 병합\n'

## Teams / 팀의 경기기록

### DATA FIELD in teams

- teamId - 경기내의 파랑팀 (100) / 경기내의 빨강팀 (200)
- win - 승 / 패 , target_variable로 사용할 변수입니다. 
- firstBlood - 최초 킬 여부
- firstTower - 최초 포탑 파괴 여부 
- firstinhibitor - 최초 억제기 파괴 여부 
- firstBaron - 최초 바론 처치 여부
- firstDragon - 최초 드래곤 처치 여부 
- firstRiftHerald - 최초 전령 처치 여부 
- towerKills - 타워 파괴 횟수
- inhibitorKills - 억제기 파괴 횟수
- baronKills - 바론 처치 횟수
- dragonKills - 드래곤 처치 횟수
- vilemawKills - (5대5 클래식 게임에 필요없음)
- riftHeraldKills - 전령 처치 횟수 
- dominionVictoryScore - (5대5 클래식 게임에 필요없음)

In [9]:
"""
게임 한 판당 team red, blue의 기록
"""
# data = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match.pkl")
# temp_team = list(data["teams"])
# #team red
# team1_df = pd.DataFrame()
# for i in tqdm_notebook(range(len(temp_team))):
#     temp_team[i][0].pop("bans",None)
#     team1 = pd.DataFrame(list(temp_team[i][0].values()),index = list(temp_team[i][0].keys())).T
#     team1_df = team1_df.append(team1)

# team1_df.index = range(len(team1_df))

# #team blue
# team2_df = pd.DataFrame()
# for i in tqdm_notebook(range(len(temp_team))):
#     temp_team[i][1].pop("bans",None)
#     team2 = pd.DataFrame(list(temp_team[i][1].values()),index = list(temp_team[i][1].keys())).T
#     team2_df = team2_df.append(team2)
    
# team2_df.index = range(len(team2_df))

# team red와 team blue, gameId 합치기
# data_team_df = pd.concat([team1_df,team2_df,data[["gameId"]]],axis=1)

# data_team_df.to_pickle("team.pkl") 

'\n게임 한 판당 team red, blue의 기록\n'

In [10]:
"""
팀 데이터 불러온 후, 팀 json 중간중간 비어있는 값 제외 
"""
# team_a_error = [] 
# team_b_error = []
# team_a = pd.DataFrame()
# team_b = pd.DataFrame()
# for i in range(len(lol_df)):
#     try:
#         team_a = team_a.append(json_normalize(lol_df["teams"].iloc[i][0]))
#         team_b = team_b.append(json_normalize(lol_df["teams"].iloc[i][1]))
#     except:
#         team_a_error.append(i)
#         team_b_error.append(i)
#         print(str(i)+"행에서 오류")
#         pass

'\n팀 데이터 불러온 후, 팀 json 중간중간 비어있는 값 제외 \n'

In [11]:
"""
승리팀, 패배팀으로 나누고 정합성 확인
"""
# for i in range(len(team_a)):
#     wf_valid = team_a["win"].iloc[i]
    
#     if team_b["win"].iloc[i] != wf_valid:
#         pass
#     else:
#         print(str(i)+"행 정합성 문제")

'\n승리팀, 패배팀으로 나누고 정합성 확인\n'

## Participants / 게임 참가자 정보

### DATA FIELD in participants

participantId': 게임 참가자id 1~10,

'teamId': 팀 아이디 RED-100, BLUE-200

'championId': 챔피언 고유 아이디,

'spell1Id': 스펠1 고유 아이디,

'spell2Id': 스펠2 고유 아이디,

'stats': 참가자의 스탯

'timeline': 해당 게임의 타임라인 데이터

### DATA FIELD in participants.stats

- stat : statistic
    - win : T(win) / F(lose) 승리 정보
    - item(0~6) : 고유 아이템1-6 아이디
    - kills : 처치 횟수
    - deaths : 죽음 횟수
    - assists : 어시스트 횟수
        - (+ kda)
    - largestKillingSpree : 최대 연속처치
    - largestMultiKill : 최대 다중킬
    - killingSprees : 죽지 않고 처치한 횟수
    - longestTimeSpentLiving : 최대 죽지 않고 살아있던 시간
    - doubleKills(triple ~ unrealkill) : 멀티킬 횟수
    - totalDamageDealt : 총 피해량
    - magicDamageDealt : 총 마법피해량
    - physicalDamageDealt : 총 물리피해량
    - trueDamageDealt :  총 고정피해량
    - largestCriticalStrike : 치명타 횟수
    - totalDamageDealtToChampions : 챔피언에게 가한 총 피해량
    - magicDamageDealtToChampions : 챔피언에게 가한 총 마법 피해량
    - physicalDamageDealtToChampions : 챔피언에게 가한 총 물리 피해량
    - trueDamageDealtToChampions : 챔피언에게 가한 총 고정 피해량
    - totalHeal : 총 회복량
    - totalUnitsHealed : 총 챔피언 자체 회복량
    - damageSelfMitigated : 총 받은피해 감소량
    - damageDealtToObjectives : 오브젝트(ex.용,바위게,바론)에 가한 피해량
    - damageDealtToTurrets : 포탑에 가한 피해량
    - visionScore : Vision score 시야 점수
    - timeCCingOthers : cc기 시간
    - totalDamageTaken : 총 받은 피해량
    - magicalDamageTaken : 총 받은 마법피해량
    - physicalDamageTaken : 총 받은 물리피해량
    - trueDamageTaken : 총 받은 고정피해량
    - goldEarned : 획득한 골드
    - goldSpent : 사용한 골드
    - turretKills : 부순 포탑 갯수
    - inhibitorKills : 억제기 부순 갯수
    - totalMinionsKilled : 총 처치한 미니언
    - neutralMinionsKilled : 총 처치한 중립 몬스터 
    - neutralMinionsKilledTeamJungle : 총 처치한 아군 중립몬스터
    - neutralMinionsKilledEnemyJungle : 총 처치한 적군 중립몬스터
    - totalTimeCrowdControlDealt : 총 사용한 군중제어기(cc)사용 시간
    - champLevel : 챔피언 레벨
    - visionWardsBoughtInGame : 제어와드 구매 횟수
    - sightWardsBoughtInGame : 시야석 구매 횟수
    - wardsPlaced : 와드 설치 횟수
    - wardsKilled : 와드 제거 횟수
    - firstBloodKill : 최초 킬 여부
    - firstBloodAssist : 최초 킬 어시스트 여부
    - firstTowerKill : 최초 포탑 파괴 여부
    - firstTowerAssist : 최초 포탑 파괴 어시스트 여부
    - firstInhibitorKill : 최초 억제기 파괴 여부
    - firstInhibitorAssist : 최초 억제기 파괴 어시스트 여부
    - 이하 정보는 값이 대부분 0이므로 사용불가
- role : 다인큐 or 솔로큐
- lane : 라인(잘못된 값 많음)

원래라면 모든 칼럼을 불러와 EDA 후 유용한 칼럼을 선정하는 게 맞지만 데이터 추출에 오랜시간이 걸려

팀 단위에서 승패를 예측하기에 유용한 칼럼을 설문조사 / 도메인 지식으로 선정

<https://forms.gle/s8XC4yr5a1381MHh8>

![](https://i.imgur.com/gBmorru.png)

9만개의 게임 개인 스탯 데이터(90000x2x10xuse_cols_count)를 추출하는 것은 굉장히 오래 걸린다. 이는 데이터 분석에 있어서 원활한 실험을 못하게 하는데, 

이를 위한 해결 방안으로 로컬 환경에서는 3분할 후 결합 방식으로, 멀티프로세싱을 진행하였다.

https://pypi.org/project/parmap/ , parmap 모듈을 이용하면 쉽게 멀티코어를 이용할 수 있다. 또한 pm_pbar 파라미터를 통해 프로세스의 tqdm같은 작업 진행률 progress bar를 볼 수 있다.

In [12]:
""" 
게임 한판당 팀 구분 없는 모든 참가자(10*90500)명의 스탯 데이터
"""
# use_cols = ["kills","deaths","totalDamageDealtToChampions","goldEarned", "visionScore","totalTimeCrowdControlDealt"] Making columns to Using stat infomation/스탯에서 칼럼으로 추출할 것 기재 
# stats_df1 = pd.DataFrame()
# for i in tqdm_notebook(range(30000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         cur_values = {f"{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(10)}
#         temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df1 = pd.concat([stats_df1, temp], axis=1, sort=False)
# stats_df1 = stats_df1.T.reset_index(drop=True)

# stats_df2 = pd.DataFrame()
# for i in tqdm_notebook(range(30000,60000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         cur_values = {f"{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(10)}
#         temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df2 = pd.concat([stats_df2, temp], axis=1, sort=False)
# stats_df2 = stats_df2.T.reset_index(drop=True)

# stats_df2 = pd.concat([stats_df1,stats_df2], ignore_index=True)

# stats_df3 = pd.DataFrame()
# for i in tqdm_notebook(range(60000,90500),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         cur_values = {f"{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(10)}
#         temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df3 = pd.concat([stats_df3, temp], axis=1, sort=False)
# stats_df3 = stats_df3.T.reset_index(drop=True)

# stats_df = pd.concat([stats_df,stats_df3], ignore_index=True)

# stats_df["gameId"] = match["gameId"]

# stats_df.info()

# stats_df.to_csv("stats.csv",index=False)

' \n게임 한판당 팀 구분 없는 모든 참가자(10*90500)명의 스탯 데이터\n'

In [13]:
"""
게임 한 판당 승리한 팀원들의 스탯
"""
# use_cols = ["kills","deaths","totalDamageDealtToChampions","goldEarned", "visionScore","totalTimeCrowdControlDealt"]
# stats_df = pd.DataFrame()
# for i in tqdm_notebook(range(30000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if (match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"win_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"win_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df = pd.concat([stats_df, temp], axis=1, sort=False)
# stats_df = stats_df.T.reset_index(drop=True)

# stats_df2 = pd.DataFrame()
# for i in tqdm_notebook(range(30000,60000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if (match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"win_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"win_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df2 = pd.concat([stats_df2, temp], axis=1, sort=False)
# stats_df2 = stats_df2.T.reset_index(drop=True)

# win_stats_df = pd.concat([stats_df,stats_df2], ignore_index=True)

# stats_df3 = pd.DataFrame()
# for i in tqdm_notebook(range(60000,90500),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if (match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"win_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"win_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df3 = pd.concat([stats_df3, temp], axis=1, sort=False)
# stats_df3 = stats_df3.T.reset_index(drop=True)

# win_stats_df = pd.concat([win_stats_df,stats_df3], ignore_index=True)

# win_stats_df["gameId"] = match["gameId"]

# win_stats_df.info()

# win_stats_df.to_csv("win_team_stats.csv",index=False)

'\n게임 한 판당 승리한 팀원들의 스탯\n'

In [14]:
"""
게임 한 판당 패배한 팀원들의 스탯
"""
# use_cols = ["kills","deaths","totalDamageDealtToChampions","goldEarned", "visionScore","totalTimeCrowdControlDealt"]
# stats_df1 = pd.DataFrame()
# for i in tqdm_notebook(range(30000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if not(match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"lose_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"lose_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df1 = pd.concat([stats_df1, temp], axis=1, sort=False)
# stats_df1 = stats_df1.T.reset_index(drop=True)

# stats_df2 = pd.DataFrame()
# for i in tqdm_notebook(range(30000,60000),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if not(match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"lose_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"lose_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df2 = pd.concat([stats_df2, temp], axis=1, sort=False)
# stats_df2 = stats_df2.T.reset_index(drop=True)

# lose_stats_df = pd.concat([stats_df1,stats_df2], ignore_index=True)

# stats_df3 = pd.DataFrame()
# for i in tqdm_notebook(range(60000,90500),desc="total"):
#     temp = pd.DataFrame()
#     for col in use_cols:
#         if not(match["participants"].iat[i][0]["stats"]["win"]):
#             cur_values = {f"lose_{col}{j+1}": match["participants"].iat[i][j]["stats"][col] for j in range(5)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#         else:
#             cur_values = {f"lose_{col}{j-4}": match["participants"].iat[i][j]["stats"][col] for j in range(5,10)}
#             temp = pd.concat([temp, pd.Series(cur_values)], axis=0, sort=False)
#     stats_df3 = pd.concat([stats_df3, temp], axis=1, sort=False)
# stats_df3 = stats_df3.T.reset_index(drop=True)

# lose_stats_df = pd.concat([lose_stats_df,stats_df3], ignore_index=True)

# lose_stats_df["gameId"] = match["gameId"]

# lose_stats_df.info()

# lose_stats_df.to_csv("lose_team_stats.csv",index=False)

'\n게임 한 판당 패배한 팀원들의 스탯\n'

![](https://i.imgur.com/SkvV8nb.png)

In [15]:
"""
match 데이터가 차지하는 램 용량이 너무 커서 전처리
"""
# match = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_ver1.pkl")
# match = match[match["gameMode"] == "CLASSIC"]
# match.drop(["gameCreation","gameMode","gameType","mapId","platformId","status.message","status.status_code"],axis=1,inplace=True)
# match['gameVersion'] = match['gameVersion'].str.split(".").map(lambda x: "".join(x))
# match.to_pickle("matchpre.pkl")

'\nmatch 데이터가 차지하는 램 용량이 너무 커서 전처리\n'

매치 데이터에서 소환사의 협곡 게임(5대5)만 분석할 것이므로 클래식 게임에 대한 데이터만 남김

승패 예측에 필요없는 칼럼 제거

gameVersion 전처리